# Vertex LLM Demo: Simplify Complex Information

This demo aims to distill critical points from text and informaton that is more consumable for non-experts to understand.  This could make information delivery more efficient more both the giver and receiver of information, for example a medical provider and a patient.

# CUJs
  * As a patient, I want to understand what this medical note means to me
  * As a provider, I want to focus on critical and objective medical diagnoses and prognoses, and expedite delivery of information to patients.
  * As a product manager, I want to understand the implications of a new technology to my product or future products.
  * and many, many more...

# What the code does

  * The code takes a text as input, such as a medical chart note or complex technical explanation.
  * It then pulls that text into a prompt, instructing the LLM to generate a  summary of the text for an individual at a more appropriate knowledge level (e.g. non-expert, patient, student).
  * The model responds with a summary, and sometimes deeper explanations if certain terms within the summary still require expounding upon.

# Instructions

To run the demo, simply run all the cells in order. The hidden cells can remain hidden and you can run the entire section at once. Note that the workspace setup will trigger a runtime restart; that is working as intended to ensure the right version of the Cloud SDK is installed.


# Limitations and ideas for extension

  * Could possibly be used in a more real-time scenario as part of a chatbot deployed by medical providers / teachers / product teams / etc..
  * The body of text currently needs to fit within the LLM's context window. We can naively solve that problem by chunking up the original body of text and generating summaries of smaller chunks.
  * Some of the input could be sensitive and directly impactful, e.g. in the case of medical information, so the summary would need to be validated prior to a patient, student, individual seeing to ensure minimal or no misinformation.

# Sample output

**Original text**

*72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE. His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema. His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout. However, his low albumin which could contribute to his edema may be due to renal losses. So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include: 1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or 5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis. There is no evidence in his history, EKG, or enzymes for current ischemia/infarct. He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated. The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake. We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.*

**Response - non-expert summary (text-bison-001 model)**

*72 year old man with heart failure due to previous heart attack, kidney disease and poor circulation is admitted to hospital with worsening symptoms. Doctors are trying to figure out what caused his condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.*

**Expanded Response - explanation of non-expert summary (text-bison-001 model)**

*72 year old man with heart failure due to previous heart attack, kidney disease and poor circulation is admitted to hospital with worsening symptoms. Doctors are trying to figure out what caused his condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.*

*A 72-year-old man with heart failure due to a previous heart attack, kidney disease, and poor circulation was admitted to the hospital with worsening symptoms. The doctors are trying to figure out what caused his condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.*

*Heart failure is a condition in which the heart cannot pump enough blood to meet the body's needs. This can lead to symptoms such as shortness of breath, fatigue, and swelling in the feet, ankles, and legs. Heart failure can be caused by a number of factors, including heart attack, high blood pressure, and diabetes.*

*Kidney disease is a condition in which the kidneys cannot function properly. This can lead to symptoms such as swelling, fatigue, and urination problems. Kidney disease can be caused by a number of factors, including high blood pressure, diabetes, and smoking.*

*Poor circulation is a condition in which the blood does not flow properly through the body. This can lead to symptoms such as coldness, numbness, and pain in the feet, ankles, and legs. Poor circulation can be caused by a number of factors, including high blood pressure, diabetes, and smoking.*

*The doctors are trying to figure out what caused the man's condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.*

*The man's treatment will depend on the cause of his condition. If his heart is not working as well as it used to, he may need surgery to repair or replace his heart. If he has another heart problem, he may need medication to treat it. If he is not taking his medication properly, he will need to be started on a new medication regimen. If he is drinking too much water and eating too much salt, he will need to be put on a diet that restricts his fluid and salt intake.*

*The man's prognosis will depend on the cause of his condition and how well he responds to treatment. If his heart is not working as well as it used to, he may have a poor prognosis. If he has another heart problem, he may also have a poor prognosis. However, if he is taking his medication properly and is not drinking too much water and eating too much salt, he may have a good prognosis.*



## Workspace Setup ##

---

Run the hidden cells to set up the workspace. Note that it will automatically restart the runtime to install the Google Cloud SDK.

---

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
# !gsutil cp gs://agravat-octo-aif-sandbox-central/llm_demo/google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl .
# !pip install google_cloud_aiplatform-1.23.0.llm.alpha.23.03.28-py2.py3-none-any.whl "shapely<2.0.0"

!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.2 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


{'status': 'ok', 'restart': True}

## LLM Setup ##

---

This section sets up an easy interface for interaction with Vertex LLM.

---

In [ ]:
## Pulled from OCTO examples.  Simplifies the calls to the GenAI models.

from google.cloud import aiplatform
from google.cloud.aiplatform_v1.types.prediction_service import PredictResponse
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

class VertexLLM:

    MODEL_TEXT_BISON_001 = "text-bison-001"
    MODEL_TEXT_BISON_ALPHA = "text-bison-alpha"

    def __init__(self, endpoint="us-central1-aiplatform.googleapis.com", project="cloud-large-language-models", location="us-central1"):
        self.client = aiplatform.gapic.PredictionServiceClient(client_options={"api_endpoint": endpoint})
        self.project = project
        self.location = location

    def predict(self, content, model=MODEL_TEXT_BISON_001, temp=0.2, max_tokens=1024, top_p=0.95, top_k=40, **kwargs):
        endpoint = self.client.endpoint_path(project=self.project, location=self.location, endpoint=model)
        instance = json_format.ParseDict({"content": content}, Value())
        params = {
                "temperature": temp,
                "maxDecodeSteps": max_tokens,
                "topP": top_p,
                "topK": top_k,
        }
        response = self.client.predict(endpoint=endpoint, instances=[instance], parameters=params, **kwargs)
        response_dict = PredictResponse.to_dict(response)
        return response_dict['predictions'][0]['content']

In [ ]:
vertex_llm = VertexLLM()

## Upload Sample Body of Text##

---

This section provides some example medical chart notes that a patient may see when provided by a provider.

---



In [ ]:
# pulled from sources such as: https://www.mtsamples.com/site/pages/browse.asp?Type=87-Office%20Notes

text_samples_med = [
"""
72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated.
The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.
We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.
"""
,"""
This 66-year-old white male was seen in my office on Month DD, YYYY. Patient was recently discharged from Doctors Hospital at Parkway after he was treated for pneumonia.
Patient continues to have severe orthopnea, paroxysmal nocturnal dyspnea, cough with greenish expectoration.
His exercise tolerance is about two to three yards for shortness of breath.
The patient stopped taking Coumadin for reasons not very clear to him. He was documented to have recent atrial fibrillation.
Patient has longstanding history of ischemic heart disease, end-stage LV systolic dysfunction, and is status post ICD implantation.
Fasting blood sugar this morning is 130.
"""
,"""
This 32-year-old with family history of premature coronary artery disease came in for evaluation of recurrent chest pain, O2 saturation at 94% with both atypical and typical features of ischemia.
The patient ruled out for myocardial infarction with serial troponins.
Nuclear stress test has been done, results of which are pending.
The patient is stable to be discharged pending the results of nuclear stress test and cardiologist's recommendations.
He will follow up with cardiologist, Dr. X, in two weeks and with his primary physician in two to four weeks.
Discharge medications will depend on results of nuclear stress test.
"""
,"""
The patient returns to our office today because of continued problems with her headaches.
She was started on Zonegran on her last visit and she states that initially she titrated upto 100 mg q.h.s.
Initially felt that the Zonegran helped, but then the pain in her head returned.
It is an area of tenderness and sensitivity in her left parietal area.
It is a very localized pain.
She takes Motrin 400 mg b.i.d., which helped.
She also had EMG/nerve conduction studies since she was last seen in our office that showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
She was also never referred to the endocrine clinic to deal with her poor diabetes control.
Her last hemoglobin A1c was 10.
"""
]
text_samples_tech_math = [
"""
In probability theory and statistics, Bayes' theorem (alternatively Bayes' law or Bayes' rule), named after Thomas Bayes, describes the probability of an event, based on prior knowledge of conditions that might be related to the event.For example, if the risk of developing health problems is known to increase with age, Bayes' theorem allows the risk to an individual of a known age to be assessed more accurately by conditioning it relative to their age, rather than simply assuming that the individual is typical of the population as a whole.
One of the many applications of Bayes' theorem is Bayesian inference, a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics, being considered by one authority as; "to the theory of probability what Pythagoras's theorem is to geometry.
"""
,"""
Compound interest is the addition of interest to the principal sum of a loan or deposit, or in other words, interest on principal plus interest. It is the result of reinvesting interest, or adding it to the loaned capital rather than paying it out, or requiring payment from borrower, so that interest in the next period is then earned on the principal sum plus previously accumulated interest. Compound interest is standard in finance and economics.
Compound interest is contrasted with simple interest, where previously accumulated interest is not added to the principal amount of the current period, so there is no compounding. The simple annual interest rate is the interest amount per period, multiplied by the number of periods per year. The simple annual interest rate is also known as the nominal interest rate (not to be confused with the interest rate not adjusted for inflation, which goes by the same name).
"""
,"""
An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.[1] A document or standard that describes how to build or use such a connection or interface is called an API specification. A computer system that meets this standard is said to implement or expose an API. The term API may refer either to the specification or to the implementation.
In contrast to a user interface, which connects a computer to a person, an application programming interface connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them.
One purpose of APIs is to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change. An API may be custom-built for a particular pair of systems, or it may be a shared standard allowing interoperability among many systems.
There are APIs for programming languages, software libraries, computer operating systems, and computer hardware. APIs originated in the 1940s, though the term did not emerge until the 1960s and 1970s. Contemporary usage of the term API often refers to web APIs,[2] which allow communication between computers that are joined by the internet. Recent developments in APIs have led to the rise in popularity of microservices, which are loosely coupled services accessed through public APIs.
"""
]

In [ ]:
print(text_samples_med[0])
print(text_samples_tech_math[0])


72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his 

## Generate Summaries ##

---

Here we create a basic summary generation prompt and print the results.

Results are OK.

---

In [ ]:
from IPython.display import Markdown

for text in text_samples_med:

  generation_prompt = """
  Generate a summary of the following text.

  Text:
  {}

  Summary:
  """.format(text)

  model = VertexLLM.MODEL_TEXT_BISON_001
  response = vertex_llm.predict(generation_prompt, model=model)

  display(Markdown("## Original text"))
  display(Markdown(text))
  display(Markdown("### Response ({} model)".format(model)))
  display(Markdown(response))
  display(Markdown("-------------------------------------"))


## Original text


72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated.
The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.
We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.


### Response (text-bison-001 model)

72-year-old man with history of CHF, chronic renal insufficiency and venous stasis presented with worsening edema and DOE. The most likely cause is worsening CHF due to poor compliance with medication and fluid overload from excessive intake.

-------------------------------------

...

## Original text


The patient returns to our office today because of continued problems with her headaches.
She was started on Zonegran on her last visit and she states that initially she titrated upto 100 mg q.h.s.
Initially felt that the Zonegran helped, but then the pain in her head returned.
It is an area of tenderness and sensitivity in her left parietal area.
It is a very localized pain.
She takes Motrin 400 mg b.i.d., which helped.
She also had EMG/nerve conduction studies since she was last seen in our office that showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
She was also never referred to the endocrine clinic to deal with her poor diabetes control.
Her last hemoglobin A1c was 10.


### Response (text-bison-001 model)

45-year-old female with poorly controlled diabetes presents with worsening headaches.
She was started on Zonegran 100 mg q.h.s., but it did not help.
She also had EMG/nerve conduction studies that showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
She was also never referred to the endocrine clinic to deal with her poor diabetes control.
Her last hemoglobin A1c was 10.

-------------------------------------

In [ ]:
from IPython.display import Markdown

for text in text_samples_tech_math:

  generation_prompt = """
  Generate a summary of the following text.

  Text:
  {}

  Summary:
  """.format(text)

  model = VertexLLM.MODEL_TEXT_BISON_001
  response = vertex_llm.predict(generation_prompt, model=model)

  display(Markdown("## Original text"))
  display(Markdown(text))
  display(Markdown("### Response ({} model)".format(model)))
  display(Markdown(response))
  display(Markdown("-------------------------------------"))


## Original text


In probability theory and statistics, Bayes' theorem (alternatively Bayes' law or Bayes' rule), named after Thomas Bayes, describes the probability of an event, based on prior knowledge of conditions that might be related to the event.For example, if the risk of developing health problems is known to increase with age, Bayes' theorem allows the risk to an individual of a known age to be assessed more accurately by conditioning it relative to their age, rather than simply assuming that the individual is typical of the population as a whole.
One of the many applications of Bayes' theorem is Bayesian inference, a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics, being considered by one authority as; "to the theory of probability what Pythagoras's theorem is to geometry."


### Response (text-bison-001 model)

Bayes' theorem is a theorem in probability theory and statistics that describes the probability of an event, based on prior knowledge of conditions that might be related to the event. It is named after Thomas Bayes, who first wrote about it in the 18th century. Bayes' theorem has many applications, including Bayesian inference, which is a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics, being considered by one authority as; "to the theory of probability what Pythagoras's theorem is to geometry."

-------------------------------------

...

## Original text


An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.[1] A document or standard that describes how to build or use such a connection or interface is called an API specification. A computer system that meets this standard is said to implement or expose an API. The term API may refer either to the specification or to the implementation.
In contrast to a user interface, which connects a computer to a person, an application programming interface connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them.
One purpose of APIs is to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change. An API may be custom-built for a particular pair of systems, or it may be a shared standard allowing interoperability among many systems.
There are APIs for programming languages, software libraries, computer operating systems, and computer hardware. APIs originated in the 1940s, though the term did not emerge until the 1960s and 1970s. Contemporary usage of the term API often refers to web APIs,[2] which allow communication between computers that are joined by the internet. Recent developments in APIs have led to the rise in popularity of microservices, which are loosely coupled services accessed through public APIs.


### Response (text-bison-001 model)

1. An application programming interface (API) is a way for two or more computer programs to communicate with each other.
2. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software.
3. An API is often made up of different parts which act as tools or services that are available to the programmer.
4. One purpose of APIs is to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change.
5. There are APIs for programming languages, software libraries, computer operating systems, and computer hardware.
6. Contemporary usage of the term API often refers to web APIs, which allow communication between computers that are joined by the internet.

-------------------------------------


## Generate Non-Expert Summaries ##

---

Here we create a summary generation prompt that aims to simplify the content for non-experts and print the results.

Results are better than the basic prompt.

---

In [ ]:
summaries = []
summaries

[]

In [ ]:
for text in text_samples_med:

  generation_prompt = """
  Generate a summary that a non-expert would understand of the following text.

  Text:
  {}

  Summary:
  """.format(text)

  model = VertexLLM.MODEL_TEXT_BISON_001
  response = vertex_llm.predict(generation_prompt, model=model)
  summaries.append(response)

  display(Markdown("## Original text"))
  display(Markdown(text))
  display(Markdown("### Response ({} model)".format(model)))
  display(Markdown(response))
  display(Markdown("-------------------------------------"))


## Original text


72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated.
The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.
We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.


### Response (text-bison-001 model)

72 year old man with heart failure due to previous heart attack, kidney disease and poor circulation is admitted to hospital with worsening symptoms.
Doctors are trying to figure out what caused his condition to get worse.
They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt.
They will do more tests to find out the cause and treat his symptoms.

-------------------------------------

...

## Original text


The patient returns to our office today because of continued problems with her headaches.
She was started on Zonegran on her last visit and she states that initially she titrated upto 100 mg q.h.s.
Initially felt that the Zonegran helped, but then the pain in her head returned.
It is an area of tenderness and sensitivity in her left parietal area.
It is a very localized pain.
She takes Motrin 400 mg b.i.d., which helped.
She also had EMG/nerve conduction studies since she was last seen in our office that showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
She was also never referred to the endocrine clinic to deal with her poor diabetes control.
Her last hemoglobin A1c was 10.


### Response (text-bison-001 model)

The patient is a woman with type 2 diabetes who is having trouble controlling her blood sugar. She also has neuropathy, which is nerve damage caused by the disease. She was prescribed Zonegran for her headaches, but it doesn't seem to be working. She will be seeing Dr. XYZ to discuss her treatment options.

-------------------------------------

In [ ]:
for text in text_samples_tech_math:

  generation_prompt = """
  Generate a summary that a non-expert would understand of the following text.

  Text:
  {}

  Summary:
  """.format(text)

  model = VertexLLM.MODEL_TEXT_BISON_001
  response = vertex_llm.predict(generation_prompt, model=model)
  summaries.append(response)

  display(Markdown("## Original text"))
  display(Markdown(text))
  display(Markdown("### Response ({} model)".format(model)))
  display(Markdown(response))
  display(Markdown("-------------------------------------"))


## Original text


In probability theory and statistics, Bayes' theorem (alternatively Bayes' law or Bayes' rule), named after Thomas Bayes, describes the probability of an event, based on prior knowledge of conditions that might be related to the event.For example, if the risk of developing health problems is known to increase with age, Bayes' theorem allows the risk to an individual of a known age to be assessed more accurately by conditioning it relative to their age, rather than simply assuming that the individual is typical of the population as a whole.
One of the many applications of Bayes' theorem is Bayesian inference, a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics, being considered by one authority as; "to the theory of probability what Pythagoras's theorem is to geometry."


### Response (text-bison-001 model)

Bayes' theorem is a mathematical formula that can be used to calculate the probability of an event happening, based on what we already know about the event. It is named after Thomas Bayes, an 18th-century mathematician who first wrote about it.
Bayes' theorem can be used in many different situations. For example, it can be used to calculate the probability that a person has a disease, based on the results of a medical test. It can also be used to calculate the probability that a sports team will win a game, based on the team's past performance.
Bayes' theorem is a very powerful tool, but it can be difficult to understand. If you are interested in learning more about Bayes' theorem, there are many resources available online and in libraries.

-------------------------------------

...

## Original text


An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.[1] A document or standard that describes how to build or use such a connection or interface is called an API specification. A computer system that meets this standard is said to implement or expose an API. The term API may refer either to the specification or to the implementation.
In contrast to a user interface, which connects a computer to a person, an application programming interface connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them.
One purpose of APIs is to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change. An API may be custom-built for a particular pair of systems, or it may be a shared standard allowing interoperability among many systems.
There are APIs for programming languages, software libraries, computer operating systems, and computer hardware. APIs originated in the 1940s, though the term did not emerge until the 1960s and 1970s. Contemporary usage of the term API often refers to web APIs,[2] which allow communication between computers that are joined by the internet. Recent developments in APIs have led to the rise in popularity of microservices, which are loosely coupled services accessed through public APIs.


### Response (text-bison-001 model)

An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.

APIs are used to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change.

There are APIs for programming languages, software libraries, computer operating systems, and computer hardware.

-------------------------------------


## Generate Explanations Based on Simplified Summaries ##

---

Here we create an explanation generation prompt that aims to expand upon the summaries for non-experts and print the results.

Results are ___.

---

In [ ]:
len(summaries)

7

In [ ]:
for summary in summaries:

  generation_prompt = """
  Generate a more detailed explanation that a non-expert would understand of the following text.

  Previous Summary:
  {}

  Explanation:
  """.format(summary)

  model = VertexLLM.MODEL_TEXT_BISON_001
  response = vertex_llm.predict(generation_prompt, model=model)

  display(Markdown("## Original text"))
  display(Markdown(summary))
  display(Markdown("### Response ({} model)".format(model)))
  display(Markdown(response))
  display(Markdown("-------------------------------------"))


## Original text

72 year old man with heart failure due to previous heart attack, kidney disease and poor circulation is admitted to hospital with worsening symptoms.
Doctors are trying to figure out what caused his condition to get worse.
They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt.
They will do more tests to find out the cause and treat his symptoms.

### Response (text-bison-001 model)

72 year old man with heart failure due to previous heart attack, kidney disease and poor circulation is admitted to hospital with worsening symptoms.
Doctors are trying to figure out what caused his condition to get worse.
They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt.
They will do more tests to find out the cause and treat his symptoms.

A 72-year-old man with heart failure due to a previous heart attack, kidney disease, and poor circulation was admitted to the hospital with worsening symptoms. The doctors are trying to figure out what caused his condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.

Heart failure is a condition in which the heart cannot pump enough blood to meet the body's needs. This can lead to symptoms such as shortness of breath, fatigue, and swelling in the feet, ankles, and legs. Heart failure can be caused by a number of factors, including heart attack, high blood pressure, and diabetes.

Kidney disease is a condition in which the kidneys cannot function properly. This can lead to symptoms such as swelling, fatigue, and urination problems. Kidney disease can be caused by a number of factors, including high blood pressure, diabetes, and smoking.

Poor circulation is a condition in which the blood does not flow properly through the body. This can lead to symptoms such as coldness, numbness, and pain in the feet, ankles, and legs. Poor circulation can be caused by a number of factors, including high blood pressure, diabetes, and smoking.

The doctors are trying to figure out what caused the man's condition to get worse. They think it could be because his heart is not working as well as it used to, or because he has another heart problem, or because he is not taking his medication properly, or because he is drinking too much water and eating too much salt. They will do more tests to find out the cause and treat his symptoms.

The man's treatment will depend on the cause of his condition. If his heart is not working as well as it used to, he may need surgery to repair or replace his heart. If he has another heart problem, he may need medication to treat it. If he is not taking his medication properly, he will need to be started on a new medication regimen. If he is drinking too much water and eating too much salt, he will need to be put on a diet that restricts his fluid and salt intake.

The man's prognosis will depend on the cause of his condition and how well he responds to treatment. If his heart is not working as well as it used to, he may have a poor prognosis. If he has another heart problem, he may also have a poor prognosis. However, if he is taking his medication properly and is not drinking too much water and eating too much salt, he may have a good prognosis.

-------------------------------------

...

## Original text

An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.

APIs are used to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change.

There are APIs for programming languages, software libraries, computer operating systems, and computer hardware.

### Response (text-bison-001 model)

An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.

APIs are used to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change.

There are APIs for programming languages, software libraries, computer operating systems, and computer hardware.

For example, a web browser might have an API that allows other programs to display web pages. This API would hide the details of how the web browser works, such as how it renders HTML and CSS, and only expose the parts that other programs would need to know, such as how to request a web page and how to display the results.

APIs are used to make it easier for developers to build new software. By providing a set of APIs, a software company can make it easier for developers to build applications that work with their products. This can help to increase the adoption of their products.

APIs are also used to make it easier for businesses to share data with each other. By providing an API, a business can make it easier for other businesses to access their data. This can help to improve collaboration and efficiency.

APIs are a powerful tool that can be used to improve the development and use of software. By providing a set of APIs, a software company or business can make it easier for developers to build new applications and for businesses to share data.

Here are some examples of how APIs are used in the real world:

* The Google Maps API allows developers to add maps and directions to their websites and applications.
* The Facebook API allows developers to integrate Facebook features into their websites and applications, such as the ability to log in with Facebook or to share content on Facebook.
* The Twitter API allows developers to access Twitter data and functionality, such as the ability to display tweets in a website or application.

APIs are a valuable tool for developers and businesses. They can make it easier to build new applications, share data, and improve collaboration.

-------------------------------------


## Generate Even Simpler Summaries ##

---

Here we create a summary generation prompt that aims to simplify the content to even a lower level and use bullets to format and print the results.

Results are generally similar to the previous attempt, and don't always include bullet points in the response.

---

In [ ]:
for text in text_samples_med:

  generation_prompt = """
  Generate a 3-5 bullet-point summary that a 6th grader would understand of the following text.

  Text:
  {}

  Summary:
  """.format(text)

  model = VertexLLM.MODEL_TEXT_BISON_001
  response = vertex_llm.predict(generation_prompt, model=model)

  display(Markdown("## Original text"))
  display(Markdown(text))
  display(Markdown("### Response ({} model)".format(model)))
  display(Markdown(response))
  display(Markdown("-------------------------------------"))


## Original text


72 year old man with h/o CHF following MI, chronic renal insufficiency and venous stasis admitted with worsening edema and DOE.
His symptoms are most consistent with incrasing CHF-biventricular-which would account for both his pulmonary congestion as well as his peripheral edema.
His renal disease is a less likely explanation for his extensive edema as his BUN/Cr have remained stable throughout.
However, his low albumin which could contribute to his edema may be due to renal losses.
So if his edema is due to CHF, why has it become gradually and now acutely worse? Possibilities include:
1) worsening LV function, 2) another MI, 3) worsening valvular disease, 4) poor compliance with medications or
5) excess salt and water intake. His ECHO today shows no change in his EF, but there is marked wall motion abnormalities with akinesis.
There is no evidence in his history, EKG, or enzymes for current ischemia/infarct.
He does have MR and TR and his valvular disease may in part account for his worsening symptoms though his estimated PA pressure is unchanged and his LA is not dilated.
The most likely precipitant of his failure is a combination of poor compliance with medication and fluid overload from excessive intake.
We will continue to investigate the possibility of a structural precipitant for his deterioration and treat his current symptoms.


### Response (text-bison-001 model)

72-year-old man with heart failure, kidney disease, and leg swelling went to the hospital.
His symptoms are most likely caused by worsening heart failure.
The possible reasons for his worsening heart failure include:
- worsening left ventricular function
- another heart attack
- worsening valvular disease
- poor compliance with medications
- excess salt and water intake
The most likely cause is poor compliance with medication and fluid overload from excessive intake.

-------------------------------------

...

## Original text


The patient returns to our office today because of continued problems with her headaches.
She was started on Zonegran on her last visit and she states that initially she titrated upto 100 mg q.h.s.
Initially felt that the Zonegran helped, but then the pain in her head returned.
It is an area of tenderness and sensitivity in her left parietal area.
It is a very localized pain.
She takes Motrin 400 mg b.i.d., which helped.
She also had EMG/nerve conduction studies since she was last seen in our office that showed severe left ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
She was also never referred to the endocrine clinic to deal with her poor diabetes control.
Her last hemoglobin A1c was 10.


### Response (text-bison-001 model)

1. The patient is a woman who has been having headaches.
2. She was started on Zonegran, but it didn't help her headaches.
3. She also has ulnar neuropathy, moderate right ulnar neuropathy, bilateral mild-to-moderate carpal tunnel and diabetic neuropathy.
4. She was referred to Dr. XYZ and will be seeing him on August 8, 2006.
5. She was also never referred to the endocrine clinic to deal with her poor diabetes control.

-------------------------------------

In [ ]:
for text in text_samples_tech_math:

  generation_prompt = """
  Generate a 3-5 bullet-point summary that a 6th grader would understand of the following text.

  Text:
  {}

  Summary:
  """.format(text)

  model = VertexLLM.MODEL_TEXT_BISON_001
  response = vertex_llm.predict(generation_prompt, model=model)

  display(Markdown("## Original text"))
  display(Markdown(text))
  display(Markdown("### Response ({} model)".format(model)))
  display(Markdown(response))
  display(Markdown("-------------------------------------"))


## Original text


In probability theory and statistics, Bayes' theorem (alternatively Bayes' law or Bayes' rule), named after Thomas Bayes, describes the probability of an event, based on prior knowledge of conditions that might be related to the event.For example, if the risk of developing health problems is known to increase with age, Bayes' theorem allows the risk to an individual of a known age to be assessed more accurately by conditioning it relative to their age, rather than simply assuming that the individual is typical of the population as a whole.
One of the many applications of Bayes' theorem is Bayesian inference, a particular approach to statistical inference. When applied, the probabilities involved in the theorem may have different probability interpretations. With Bayesian probability interpretation, the theorem expresses how a degree of belief, expressed as a probability, should rationally change to account for the availability of related evidence. Bayesian inference is fundamental to Bayesian statistics, being considered by one authority as; "to the theory of probability what Pythagoras's theorem is to geometry."


### Response (text-bison-001 model)

Bayes' theorem is a mathematical formula that can be used to calculate the probability of an event happening, based on what we already know about the event.
For example, if we know that the risk of developing health problems increases with age, Bayes' theorem can be used to calculate the risk of an individual developing health problems, based on their age.
Bayes' theorem is used in a variety of fields, including probability theory, statistics, and medical diagnosis.
It is considered to be one of the most important theorems in probability theory.

-------------------------------------

...

## Original text


An application programming interface (API) is a way for two or more computer programs to communicate with each other. It is a type of software interface, offering a service to other pieces of software.[1] A document or standard that describes how to build or use such a connection or interface is called an API specification. A computer system that meets this standard is said to implement or expose an API. The term API may refer either to the specification or to the implementation.
In contrast to a user interface, which connects a computer to a person, an application programming interface connects computers or pieces of software to each other. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software. An API is often made up of different parts which act as tools or services that are available to the programmer. A program or a programmer that uses one of these parts is said to call that portion of the API. The calls that make up the API are also known as subroutines, methods, requests, or endpoints. An API specification defines these calls, meaning that it explains how to use or implement them.
One purpose of APIs is to hide the internal details of how a system works, exposing only those parts a programmer will find useful and keeping them consistent even if the internal details later change. An API may be custom-built for a particular pair of systems, or it may be a shared standard allowing interoperability among many systems.
There are APIs for programming languages, software libraries, computer operating systems, and computer hardware. APIs originated in the 1940s, though the term did not emerge until the 1960s and 1970s. Contemporary usage of the term API often refers to web APIs,[2] which allow communication between computers that are joined by the internet. Recent developments in APIs have led to the rise in popularity of microservices, which are loosely coupled services accessed through public APIs.


### Response (text-bison-001 model)

1. An API is a way for two or more computer programs to communicate with each other.
2. It is not intended to be used directly by a person (the end user) other than a computer programmer who is incorporating it into the software.
3. APIs are often made up of different parts which act as tools or services that are available to the programmer.
4. APIs originated in the 1940s, though the term did not emerge until the 1960s and 1970s.

-------------------------------------